### Links
* https://community.plotly.com/t/announcing-plotly-py-4-8-plotly-express-support-for-wide-and-mixed-form-data-plus-a-pandas-backend/40048
* https://plotly.com/python/styling-plotly-express/
* https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import pandas as pd
import numpy as np

from darts import TimeSeries
from darts.models import  AutoARIMA
                          

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.io as pio


pio.templates.default = "simple_white"

px.defaults.template = "ggplot2"
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody
px.defaults.width = 700
px.defaults.height = 400

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [ ]:
df = pd.read_csv("../../data/later/profile_growth.csv")
df['Date'] = pd.to_datetime(df['Date'])
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, 'Date', 'Followers')

In [ ]:
# Create figures in Express
figure1 = px.line(df, x="Date", y=["Followers"], title='Followers')
figure2 = px.line(df,
                  x="Date",
                  y=["Impressions", "Reach"],
                  title='Impressions & Reach')


# For as many traces that exist per Express figure, get the traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into it's traces
figure1_traces = []
figure2_traces = []
for trace in range(len(figure1["data"])):
    figure1_traces.append(figure1["data"][trace])
for trace in range(len(figure2["data"])):
    figure2_traces.append(figure2["data"][trace])

In [ ]:
model = AutoARIMA()
model.fit(series)
forecast = model.predict(3)
series_df = series.pd_dataframe().reset_index()#.set_index('component')
forecast_df = forecast.pd_dataframe().reset_index()

In [ ]:
# Create traces
forecast_fig = go.Figure()
forecast_fig.add_trace(go.Scatter( x=series_df['Date'], y=series_df['Followers'],
                    mode='lines+markers',
                    name='ToDate'))
forecast_fig.add_trace(go.Scatter(x=forecast_df['Date'], y=forecast_df['Followers'],
                    mode='lines+markers',
                    name='Forecast'))

In [ ]:
# Create a 1x2 subplot
this_figure = sp.make_subplots(rows=2,
                               cols=1,
                               subplot_titles=("Followers", "Impressions & Reach"),
                               row_heights=[0.3, 0.7])

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in figure1_traces:
    this_figure.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    this_figure.append_trace(traces, row=2, col=1)
    
# Update xaxis properties
#this_figure.update_xaxes(title_text="Date", row=1, col=1)

# Update yaxis properties
this_figure.update_yaxes(title_text="Count", visible=False, fixedrange=True, row=1, col=1)
this_figure.update_yaxes(title_text="Count", row=2, col=1)

# hide and lock down axes


this_figure.update_layout(showlegend=False,
                          height=700,
                          width=700,
                          title_text="Profile Growth & Discovery")
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("JupyterDash Demo"),
     html.Div([dcc.Graph(figure=this_figure)]),
     html.Div([dcc.Graph(figure=forecast_fig)])
     
    ])

# Run app and display result inline in the notebook
app.run_server(mode='external')